In [1]:
import pickle

import pandas as pd

import numpy as np

from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

In [2]:
iris = datasets.load_iris()
data = pd.DataFrame(
    {
        "sepalLength": iris.data[:, 0],
        "sepalWidth": iris.data[:, 1],
        "petalLength": iris.data[:, 2],
        "petalWidth": iris.data[:, 3],
        "species": iris.target,
    }
)
data.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
X = data[["sepalLength", "sepalWidth", "petalLength", "petalWidth"]]  # Features
y = data["species"]  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3
)  # 70% training and 30% test

type(X_train)

pandas.core.frame.DataFrame

In [ ]:
# Create a Gaussian Classifier
model = RandomForestClassifier(n_estimators=100)

# Train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
print(X_train)

In [ ]:
model.predict([[5, 3, 1.6, 0.2]])

In [ ]:
# save the model to disk
filename = "../models/iris-model.pkl"
pickle.dump(model, open(filename, "wb"))

# some time later...

# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)